In [1]:
import pandas as pd
import numpy as np

import cmsdials
from cmsdials import Dials
from cmsdials.auth.bearer import Credentials
from cmsdials.filters import (
    FileIndexFilters,
    LumisectionHistogram1DFilters,
    LumisectionHistogram2DFilters,
    LumisectionFilters,
    RunFilters
)

creds = Credentials.from_creds_file()
dials = Dials(creds, nthreads=2)

In [2]:
import matplotlib.pyplot as plt
import mplhep as hep
import awkward as ak

In [3]:
hep.style.use("CMS")

In [4]:
# dials.run.list_all(RunFilters(min_run_number=360392, max_run_number=365000), max_pages=5)
dials.run.list_all(RunFilters(min_run_number=379390, max_run_number=399000),max_pages=10)

PaginatedRunList(count=12, next=None, previous=None, results=[Run(run_number=379390, ls_count=238), Run(run_number=379391, ls_count=670), Run(run_number=379413, ls_count=8), Run(run_number=379415, ls_count=61), Run(run_number=379416, ls_count=233), Run(run_number=379420, ls_count=466), Run(run_number=379425, ls_count=235), Run(run_number=379433, ls_count=53), Run(run_number=379441, ls_count=1), Run(run_number=379442, ls_count=291), Run(run_number=379443, ls_count=11), Run(run_number=379454, ls_count=446)])

In [ ]:
Ls1Dhist = LumisectionHistogram1DFilters(title="PixelPhase1/Tracks/PXBarrel/charge_PXLayer_1",min_entries= 1000,min_run_number=379_390)

data = dials.h1d.list_all(Ls1Dhist,max_pages=100)


# Using Awkward Arrays

In [ ]:
# For plotting I think using this library might be better
a1 = ak.Array([i.dict() for i in data.results])

In [ ]:
a1.type

In [ ]:
a1['edges'] = np.linspace(a1.x_min,a1.x_max,num=101).to_numpy().T

run = a1[a1.run_number == 379391]

In [ ]:
def make_histo(data , ax, **kwargs) -> None:
    ax.set_title(kwargs.pop("title",None))
    ax.set_xlabel(kwargs.pop("xlabel",None))
    ax.set_ylabel(kwargs.pop("ylabel",None))
    for i in range(len(data)):
        ax.stairs(values = data.data[i], edges = data.edges[i] , **kwargs)


In [ ]:
fig,ax = plt.subplots()
make_histo(run[::20],ax,
           title = "Pixel Barrel Layer 1", xlabel= "Charge [Electrons]")

# Trying with Pandas

In [ ]:
# We can convert it to a table using pandas

df = pd.DataFrame([x.__dict__ for x in data.results])
df


In [ ]:
import numpy as np
edges = np.linspace(df.x_min,df.x_max,101).T
df.loc[:,'edges'] = list(edges)
df_run = df[df.run_number == 379391]
df_run

In [ ]:
fig,ax = plt.subplots()
make_histo(df.iloc[::20].reset_index(),ax,
           title = "Pixel Barrel Layer 1", xlabel= "Charge [Electrons]")

    